<a href="https://colab.research.google.com/github/ameschini2206/DSII-Primera_Entrega_Alejandro_Meschini/blob/main/Primera_Entrega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Proyecto DATA SCIENCE II - ALEJANDRO MESCHINI
## Primera Entrega  
### Archivo: Primera_Entrega.ipynb

---



In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

url_proyecto:str = r'https://raw.githubusercontent.com/ameschini2206/DSII-Primera_Entrega_Alejandro_Meschini/refs/heads/main/Venta%20de%20autoelevadores%20AV3.csv'
df = pd.read_csv(url_proyecto)
df.head()

,ID_venta,Fecha_venta,Marca,Tipo,Capacidad_kg,Altura_torre_mm,Provincia,Cliente_tipo,Año_fabricación,Estado,Método_pago,Vendedor,Horas_uso,Precio_USD
0,VENT00822,2022-01-07,Linde,eléctrico,5000,3000,Santa Fe,particular,2019.0,usado,contado,Grupo Industrial SA,2939.574817,49656.58
1,VENT00209,2022-01-08,Hyster,gas,1800,3000,Santa Fe,particular,2015.0,reacondicionado,contado,Potencia SRL,5014.133313,19701.25
2,VENT00073,2022-01-09,Yale,eléctrico,3500,4700,Buenos Aires,empresa logística,2017.0,nuevo,contado,Grupo Industrial SA,2754.485177,54889.31
3,VENT01169,2022-01-10,Linde,eléctrico,1800,4000,Santa Fe,distribuidor,2019.0,usado,contado,Forklift Solutions,6395.600622,12000.00
4,VENT00129,2022-01-11,Hyster,eléctrico,1800,4000,Buenos Aires,NaN,2023.0,usado,leasing,Forklift Solutions,1564.786242,18145.26


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250 entries, 0 to 1249
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID_venta         1250 non-null   object 
 1   Fecha_venta      1250 non-null   object 
 2   Marca            1250 non-null   object 
 3   Tipo             1250 non-null   object 
 4   Capacidad_kg     1250 non-null   int64  
 5   Altura_torre_mm  1250 non-null   int64  
 6   Provincia        1163 non-null   object 
 7   Cliente_tipo     1153 non-null   object 
 8   Año_fabricación  1148 non-null   float64
 9   Estado           1250 non-null   object 
 10  Método_pago      1146 non-null   object 
 11  Vendedor         1250 non-null   object 
 12  Horas_uso        1163 non-null   float64
 13  Precio_USD       1250 non-null   float64
dtypes: float64(3), int64(2), object(9)
memory usage: 136.8+ KB


In [10]:
df.shape

(1250, 14)

In [11]:
df.dtypes

,0
ID_venta,object
Fecha_venta,object
Marca,object
Tipo,object
Capacidad_kg,int64
Altura_torre_mm,int64
Provincia,object
Cliente_tipo,object
Año_fabricación,float64
Estado,object
